# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.__version__

'2.5.1'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values # write down here why we are using 3:-1 why these columns
y = dataset.iloc[:, -1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2]=le.fit_transform(X[:,2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
transformed=ct.fit_transform(X)
X=pd.DataFrame(transformed,columns=ct.get_feature_names_out())

In [9]:
print(X)

     encoder__x1_France encoder__x1_Germany encoder__x1_Spain remainder__x0  \
0                   1.0                 0.0               0.0           619   
1                   0.0                 0.0               1.0           608   
2                   1.0                 0.0               0.0           502   
3                   1.0                 0.0               0.0           699   
4                   0.0                 0.0               1.0           850   
...                 ...                 ...               ...           ...   
9995                1.0                 0.0               0.0           771   
9996                1.0                 0.0               0.0           516   
9997                1.0                 0.0               0.0           709   
9998                0.0                 1.0               0.0           772   
9999                1.0                 0.0               0.0           792   

     remainder__x2 remainder__x3 remainder__x4 rema

### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=41,test_size=0.2)

### Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [12]:
class NN(nn.Module):
    def __init__(self,input_size=X.shape[1],classes=1):
        super().__init__()
        self.fc1=nn.Linear(input_size,128)
        self.fc2=nn.Linear(128,64)
        self.out=nn.Linear(64,1)
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.sigmoid(self.out(x))
        return x
model=NN()

### Adding the input layer and the first hidden layer

In [13]:
#Adding hyperparameters
lr=3e-4
batch_size=64
epochs=10
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)
y_train=y_train.float()
y_test=y_test.float()

### Losses and Optimizer

In [14]:
loss_fn=nn.BCELoss()
optimizer=optim.Adam(model.parameters(),lr=lr)
losses=[]

### Adding the output layer

In [15]:
for i in range(101):
    y_pred=model.forward(X_train)
    y_pred=y_pred.squeeze(1)
    loss=loss_fn(y_pred,y_train)
    if i%10==0:print(f"Epoch = {i} Loss = {loss.detach().numpy()}")
    losses.append(loss.detach().numpy())
    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch = 0 Loss = 0.6598939299583435
Epoch = 10 Loss = 0.6071957945823669
Epoch = 20 Loss = 0.5639377236366272
Epoch = 30 Loss = 0.5315409302711487
Epoch = 40 Loss = 0.509679913520813
Epoch = 50 Loss = 0.4945113956928253
Epoch = 60 Loss = 0.48198941349983215
Epoch = 70 Loss = 0.4707539677619934
Epoch = 80 Loss = 0.4608076810836792
Epoch = 90 Loss = 0.451928049325943
Epoch = 100 Loss = 0.44411492347717285


## Part 3 - Training the ANN

### Compiling the ANN

### Training the ANN on the Training set

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Extra**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [21]:
x_test=torch.FloatTensor([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])
with torch.no_grad():
    a = model(x_test)
if a==torch.FloatTensor([[0.]]):
    print("Goodbye")
else:
    print("WLCM")

Goodbye


### Predicting the Test set results

In [17]:
with torch.no_grad():
    y_eval=model.forward(X_test)
    y_eval=y_eval.squeeze(1)
    loss=loss_fn(y_eval,y_test)
y_eval=y_eval.detach().numpy()
y_eval=(y_eval>0.5).astype(int)
y_test=y_test.detach().numpy()

### Making the Confusion Matrix

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_eval))
print(f"accuracy = {accuracy_score(y_test,y_eval)*100} %")

[[1592    5]
 [ 380   23]]
accuracy = 80.75 %


write down about precision recall f1-score, why is it better than just accuracy, what are some other interesting metrics u can find